In [1]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
%run ./'01 load.ipynb'

ValueError: Length mismatch: Expected axis has 16 elements, new values have 6 elements

In [3]:
country = 'Germany'

In [4]:
df = df[df['country'] == country]

KeyError: 'country'

In [ ]:
df['growth'] = df['confirmed'].diff().fillna(0)

In [ ]:
curr = df.tail(1)['date'].dt.strftime('%d. %b, %H:%M Uhr').values[0]

In [ ]:
# Newest data
df.tail(10)

In [ ]:
# Load predictions from 18th march
prediction = pd.read_csv('20200318 Predict Corona - Sheet1.csv')
prediction['predict'] = prediction['predict'].str.replace(',','').astype('str')
prediction['date'] = prediction['date'] + ' 23:59:59'
prediction['date'] = pd.to_datetime(prediction['date'])
prediction['date_string'] = prediction['date'].dt.strftime('%d. %b')

In [ ]:
df = pd.merge(left=df, right=prediction[['date', 'predict']], on='date', how='left')

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(2,1,1)
ax1.grid(True)
ax1.set_yscale('log')
ax1.plot(df['date_string'][:-1], df['confirmed'][:-1], color = 'blue', linewidth=4)
ax1.plot(df['date_string'][-2:], df['confirmed'][-2:], color = 'blue', linewidth=1)
ax1.plot(df['date_string'], df['predict'], color = 'red', linewidth=2)
ax2 = ax1.twinx()
ax2.plot(df['date_string'][:-1], df['confirmed'][:-1], color = 'green', linewidth=4)
ax2.plot(df['date_string'][-2:], df['confirmed'][-2:], color = 'green', linewidth=1)
ax2.plot(df['date_string'], df['predict'], color = 'red', linewidth=2)
ax2.plot(df['date_string'][:-1], df['growth'][:-1], color = 'orange', linewidth=4)
ax2.plot(df['date_string'][-2:], df['growth'][-2:], color = 'orange', linewidth=1)
fig.legend([
    'Bestätigte Fälle (log)',
    'Bestätigte Fälle (log) / ' + curr,
    'Vorhersage vom 18.03. (log)',
    'Bestätigte Fälle (abs)',
    'Bestätigte Fälle (abs) / ' + curr,
    'Vorhersage vom 18.03. (abs)',
    'Zuwachs',
    'Zuwachs / ' + curr], loc='upper left')
ax1.set_xticklabels(labels=df['date_string'], rotation=45)
ax3 = fig.add_subplot(2,1,2)
ax3.plot(df['date_string'][:-1], df['growth'][:-1], color = 'orange', linewidth=4)
ax3.plot(df['date_string'][-2:], df['growth'][-2:], color = 'orange', linewidth=1)
ax1.set_xticklabels([])
ax3.set_xticklabels(labels=df['date_string'], rotation=45)
labels = ax3.xaxis.get_ticklabels()
for label in labels:
    label.set_visible(False)
for label in labels[::7]:
    label.set_visible(True)
fig.tight_layout()
fig.show()